[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/RichmondAlake/agent_memory_course/blob/main/memory_bank/memory_augmented_agent_with_local_memory.ipynb)


In [1]:
! pip install -qU google-adk

In [2]:
import asyncio
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService, Session
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory import InMemoryMemoryService 
from google.adk.runners import Runner
from google.adk.tools import load_memory
from google.genai.types import Content, Part
from google.adk.tools.tool_context import ToolContext
from google.adk.tools import FunctionTool



/Users/richmondalake/miniconda3/envs/agent_memory_course/lib/python3.11/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


In [3]:
# --- Constants ---
APP_NAME = "memory_example_app"
USER_ID = "mem_user"
MODEL = "gemini-2.0-flash" # Use a valid model

In [ ]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [ ]:
set_env_securely("GOOGLE_API_KEY", "Enter your GOOGLE_API_KEY: ")

## Data Ingestion and Setup

In [5]:
import sys

# Method 1: Add the project root directory to Python path
project_root = "/Users/richmondalake/Desktop/Projects/open_source/agent_memory_course"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utilities.pdf_chunker import ingest_pdf_and_chunk

url = "https://arxiv.org/pdf/2404.13501"

# Ingest and chunk the PDF
chunks = ingest_pdf_and_chunk(url)

Loading PDF with LangChain...
Chunking text...
Successfully created 197 chunks


In [6]:
# --- Services ---
# Services must be shared across runners to share state and memory
# Use in-memory for demo
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService() 
artifact_service = InMemoryArtifactService()

In [7]:
def create_pdf_query_tool_with_chunks(pdf_chunks):
    """Create a PDF query tool with access to the chunks."""
    
    def pdf_query_func(query: str) -> str:
        """Query PDF content."""
        try:
            if not pdf_chunks:
                return "No PDF content available."
            
            # Search through chunks
            relevant_chunks = []
            query_lower = query.lower()
            
            for chunk in pdf_chunks:
                content = chunk['value']['content'].lower()
                if any(term in content for term in query_lower.split()):
                    relevant_chunks.append(chunk['value']['content'])
            
            if relevant_chunks:
                result = "\n\n".join(relevant_chunks[:3])
                return f"Found relevant PDF content:\n\n{result}"
            else:
                return f"No relevant content found for: {query}"
                
        except Exception as e:
            return f"Error searching PDF: {str(e)}"
    
    return FunctionTool(func=pdf_query_func)


In [8]:

# Create the PDF query tool with your chunks
pdf_query_tool = create_pdf_query_tool_with_chunks(chunks)


In [12]:
# Update the agent to use the corrected tool
pdf_memory_agent = LlmAgent(
    model=MODEL,
    name="PDFMemoryAgent",
    instruction="You are an AI assistant that can query PDF content and use memory. "
                "Use the 'pdf_query_func' to search through PDF content. "
                "Use the 'load_memory' tool to recall past conversations. "
                "When a user asks about PDF content, use the pdf_query_func tool.",
    tools=[load_memory, pdf_query_tool]
)

In [13]:
async def run_pdf_scenario():
    """Run scenario with three specific questions about the paper."""
    
    # Create runner
    runner = Runner(
        agent=pdf_memory_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service,
        artifact_service=artifact_service
    )
    
    # Create session for querying
    session_id = "pdf_query_session"
    await runner.session_service.create_session(
        app_name=APP_NAME, 
        user_id=USER_ID, 
        session_id=session_id
    )
    
    # Define three questions about the paper
    questions = [
        "What is the main contribution and purpose of this research paper?",
        "What methodology or approach does the paper use for their experiments?",
        "What are the key findings and results presented in this paper?"
    ]
    
    # Ask each question and collect responses
    for i, question in enumerate(questions, 1):
        print(f"\n--- Question {i}: {question} ---")
        
        query_input = Content(
            parts=[Part(text=question)], 
            role="user"
        )
        
        async for event in runner.run_async(
            user_id=USER_ID, 
            session_id=session_id, 
            new_message=query_input
        ):
            if event.is_final_response() and event.content and event.content.parts:
                print(f"Answer {i}: {event.content.parts[0].text}")
                print("-" * 80)
    
    # Add this session to memory for future recall
    completed_session = await runner.session_service.get_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=session_id
    )
    await memory_service.add_session_to_memory(completed_session)
    print("\n✓ Session added to memory for future recall.")
    
    # Test memory recall in a new session
    print("\n--- Testing Memory Recall ---")
    memory_session_id = "memory_test_session"
    await runner.session_service.create_session(
        app_name=APP_NAME, 
        user_id=USER_ID, 
        session_id=memory_session_id
    )
    
    memory_query = Content(
        parts=[Part(text="Can you summarize what we discussed about the research paper's main contributions?")], 
        role="user"
    )
    
    async for event in runner.run_async(
        user_id=USER_ID, 
        session_id=memory_session_id, 
        new_message=memory_query
    ):
        if event.is_final_response() and event.content and event.content.parts:
            print(f"Memory Recall Response: {event.content.parts[0].text}")


In [14]:
# Run the PDF scenario with three questions
await run_pdf_scenario()



--- Question 1: What is the main contribution and purpose of this research paper? ---


Answer 1: The paper is a comprehensive survey on the memory mechanisms of Large Language Model (LLM) based agents. It addresses the gap in the existing literature by providing a systematic review, summarizing and comparing different memory mechanisms, and abstracting common design patterns. The purpose is to discuss the "what" and "why" of memory in LLM-based agents, review studies on designing and evaluating memory modules, present agent applications, analyze limitations, and suggest future directions.

--------------------------------------------------------------------------------

--- Question 2: What methodology or approach does the paper use for their experiments? ---


Answer 2: The paper is a survey, so the methodology involves a systematic review of existing studies on memory mechanisms in LLM-based agents. The approach includes identifying, categorizing, and comparing different memory designs and evaluation methods used in previous research. They also analyze applications of these memory modules and discuss limitations and future research directions based on the reviewed literature.

--------------------------------------------------------------------------------

--- Question 3: What are the key findings and results presented in this paper? ---


Answer 3: Since this is a survey paper, the key findings and results are a synthesis of the existing literature. The paper likely identifies common and effective design patterns for memory mechanisms in LLM-based agents, summarizes different approaches to designing and evaluating memory modules, highlights the role of memory in various agent applications, and points out the limitations of current memory mechanisms, suggesting potential future research directions.

--------------------------------------------------------------------------------

✓ Session added to memory for future recall.

--- Testing Memory Recall ---


Memory Recall Response: The paper is a survey that provides a systematic review of memory mechanisms in Large Language Model (LLM) based agents. It summarizes and compares different memory mechanisms, abstracts common design patterns, discusses the "what" and "why" of memory in LLM-based agents, reviews studies on designing and evaluating memory modules, presents agent applications, analyzes limitations, and suggests future directions. The key findings are a synthesis of existing literature, identifying effective design patterns, summarizing approaches to memory module design and evaluation, highlighting the role of memory in applications, and suggesting future research directions.

